In [2]:
!pip install yfinance

Loading main data

In [3]:
import yfinance as yf
import numpy as np
import pandas as pd
import tensorflow as tf

In [4]:
data = yf.download("GOOGL" , start = "2018-01-01" , interval = '1d')

[*********************100%***********************]  1 of 1 completed


In [5]:
data.head(3)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-01-02,1053.020020,1075.979980,1053.020020,1073.209961,1073.209961,1588300
2018-01-03,1073.930054,1096.099976,1073.430054,1091.520020,1091.520020,1565900
2018-01-04,1097.089966,1104.079956,1094.260010,1095.760010,1095.760010,1302600


In [6]:
# Sort the data points based on indexes just for confirmation 
data.sort_index(inplace = True)

In [7]:
# Remove any duplicate index 
data = data.loc[~data.index.duplicated(keep='first')]

In [8]:
data.tail(3)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2021-10-29,2901.560059,2964.840088,2892.989990,2960.919922,2960.919922,2166000
2021-11-01,2960.919922,2960.919922,2864.510010,2869.939941,2869.939941,2342700
2021-11-02,2891.590088,2932.179932,2886.709961,2919.610107,2919.610107,679401


In [9]:
data.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-01-02,1053.020020,1075.979980,1053.020020,1073.209961,1073.209961,1588300
2018-01-03,1073.930054,1096.099976,1073.430054,1091.520020,1091.520020,1565900
2018-01-04,1097.089966,1104.079956,1094.260010,1095.760010,1095.760010,1302600
2018-01-05,1103.449951,1113.579956,1101.800049,1110.290039,1110.290039,1512500
2018-01-08,1111.000000,1119.160034,1110.000000,1114.209961,1114.209961,1232200


In [10]:
# Check for missing values 
data.isnull().sum()

Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [11]:
# Get the statistics of the data
data.describe()

,Open,High,Low,Close,Adj Close,Volume
count,967.000000,967.000000,967.000000,967.000000,967.000000,9.670000e+02
mean,1511.258841,1526.704124,1496.258079,1512.289069,1512.289069,1.785230e+06
std,523.725716,527.011324,520.536491,524.379899,524.379899,8.396355e+05
min,984.320007,1012.119995,977.659973,984.669983,984.669983,4.656000e+05
25%,1139.424988,1149.955017,1126.390015,1140.405029,1140.405029,1.245300e+06
50%,1265.739990,1275.750000,1252.119995,1264.459961,1264.459961,1.571600e+06
75%,1729.010010,1752.375000,1716.250000,1734.360046,1734.360046,2.067150e+06
max,2960.919922,2973.000000,2897.669922,2960.919922,2960.919922,6.658900e+06


Understanding Trends with in the Data

In [12]:
import plotly.graph_objects as go

In [13]:
# Check the trend in Closing Values 
fig = go.Figure()

fig.add_trace(go.Scatter(x = data.index , y = data['Close'] , mode = 'lines'))
fig.update_layout(height = 500 , width = 900, 
                  xaxis_title='Date' , yaxis_title='Close')
fig.show()

In [14]:
# Check the trend in Volume Traded
fig = go.Figure()

fig.add_trace(go.Scatter(x = data.index , y = data['Volume'] , mode = 'lines'))
fig.update_layout(height = 500 , width = 900, 
                  xaxis_title='Date' , yaxis_title='Volume')
fig.show()

Data Preparation

In [15]:
from sklearn.preprocessing import MinMaxScaler 
import pickle 
from tqdm.notebook import tnrange

In [16]:
# Filter only required data 
data = data[['Close' , 'Volume']]
data.head(3)

,Close,Volume
Date,,
2018-01-02,1073.209961,1588300
2018-01-03,1091.520020,1565900
2018-01-04,1095.760010,1302600


Scrapping extra information

In [17]:
import requests 

response = requests.get('https://www.alphavantage.co/query?function=RSI&symbol=GOOGL&interval=daily&time_period=5&series_type=close&apikey=43T9T17VCV2ME4SM') 
response = response.json()

In [18]:
response.keys()

dict_keys(['Meta Data', 'Technical Analysis: RSI'])

In [19]:
rsi_data = pd.DataFrame.from_dict(response['Technical Analysis: RSI'] , orient='index')

In [20]:
rsi_data.head()

,RSI
2004-08-26,72.7683
2004-08-27,64.2686
2004-08-30,47.8388
2004-08-31,49.2490
2004-09-01,41.0745


In [21]:
rsi_data = rsi_data[rsi_data.index >= '2018-01-01']

In [22]:
rsi_data['RSI'] = rsi_data['RSI'].astype(np.float64)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [23]:
rsi_data.head()

,RSI
2018-01-02,65.4310
2018-01-03,78.3686
2018-01-04,80.4830
2018-01-05,86.2429
2018-01-08,87.4882


In [24]:
data = data.merge(rsi_data, left_index=True, right_index=True, how='inner')

In [25]:
data.head()

,Close,Volume,RSI
2018-01-02,1073.209961,1588300,65.4310
2018-01-03,1091.520020,1565900,78.3686
2018-01-04,1095.760010,1302600,80.4830
2018-01-05,1110.290039,1512500,86.2429
2018-01-08,1114.209961,1232200,87.4882


In [26]:
# Confirm the Testing Set length 
test_length = data[(data.index >= '2020-09-01')].shape[0]

In [27]:
def CreateFeatures_and_Targets(data, feature_length):
    X = []
    Y = []

    for i in tnrange(len(data) - feature_length): 
        X.append(data.iloc[i : i + feature_length,:].values)
        Y.append(data["Close"].values[i+feature_length])

    X = np.array(X)
    Y = np.array(Y)

    return X , Y

In [28]:
X , Y = CreateFeatures_and_Targets(data , 32)

  0%|          | 0/934 [00:00<?, ?it/s]

In [29]:
# Check the shapes
X.shape , Y.shape

((934, 32, 3), (934,))

In [30]:
Xtrain , Xtest , Ytrain , Ytest = X[:-test_length] , X[-test_length:] , Y[:-test_length] , Y[-test_length:]

In [31]:
# Check Training Dataset Shape 
Xtrain.shape , Ytrain.shape

((639, 32, 3), (639,))

In [32]:
# Check Testing Dataset Shape
Xtest.shape , Ytest.shape

((295, 32, 3), (295,))

In [33]:
# Create a Scaler to Scale Vectors with Multiple Dimensions 
class MultiDimensionScaler():
    def __init__(self):
        self.scalers = []

    def fit_transform(self , X):
        total_dims = X.shape[2]
        for i in range(total_dims):
            Scaler = MinMaxScaler()
            X[:, :, i] = Scaler.fit_transform(X[:,:,i])
            self.scalers.append(Scaler)
        return X

    def transform(self , X):
        for i in range(X.shape[2]):
            X[:, :, i] = self.scalers[i].transform(X[:,:,i])
        return X 

In [34]:
Feature_Scaler = MultiDimensionScaler()
Xtrain = Feature_Scaler.fit_transform(Xtrain)
Xtest = Feature_Scaler.transform(Xtest)

In [35]:
Target_Scaler = MinMaxScaler()
Ytrain = Target_Scaler.fit_transform(Ytrain.reshape(-1,1))
Ytest = Target_Scaler.transform(Ytest.reshape(-1,1))

In [36]:
def save_object(obj , name : str):
    pickle_out = open(f"{name}.pck","wb")
    pickle.dump(obj, pickle_out)
    pickle_out.close()

def load_object(name : str):
    pickle_in = open(f"{name}.pck","rb")
    data = pickle.load(pickle_in)
    return data

In [37]:
# Save your objects for future purposes 
save_object(Feature_Scaler , "Feature_Scaler")
save_object(Target_Scaler , "Target_Scaler")

Model Building

In [38]:
from tensorflow.keras.callbacks import ModelCheckpoint , ReduceLROnPlateau

save_best = ModelCheckpoint("best_weights.h5", monitor='val_loss', save_best_only=True, save_weights_only=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.25,patience=5, min_lr=0.00001,verbose = 1)

In [39]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , Dropout , LSTM , Bidirectional , BatchNormalization

model = Sequential()

model.add(Bidirectional(LSTM(512 ,return_sequences=True , recurrent_dropout=0.1, input_shape=(32, 3))))
model.add(LSTM(256 ,recurrent_dropout=0.1))
model.add(Dropout(0.3))
model.add(Dense(64 , activation='elu'))
model.add(Dropout(0.3))
model.add(Dense(32 , activation='elu'))
model.add(Dense(1 , activation='linear'))

In [40]:
optimizer = tf.keras.optimizers.SGD(learning_rate = 0.002)
model.compile(loss='mse', optimizer=optimizer)

In [42]:
history = model.fit(Xtrain, Ytrain,
            epochs=10,
            batch_size = 1,
            verbose=1,
            shuffle=False ,
            validation_data=(Xtest , Ytest),
            callbacks=[reduce_lr , save_best])

Epoch 1/10
639/639 [==============================] - 154s 240ms/step - loss: 0.0092 - val_loss: 0.8890
Epoch 2/10
639/639 [==============================] - 151s 236ms/step - loss: 0.0111 - val_loss: 0.7490
Epoch 3/10
639/639 [==============================] - 146s 229ms/step - loss: 0.0084 - val_loss: 0.6401
Epoch 4/10
639/639 [==============================] - 143s 224ms/step - loss: 0.0068 - val_loss: 0.5746
Epoch 5/10
639/639 [==============================] - 146s 229ms/step - loss: 0.0054 - val_loss: 0.5170
Epoch 6/10
639/639 [==============================] - 150s 234ms/step - loss: 0.0054 - val_loss: 0.3981
Epoch 7/10
639/639 [==============================] - 150s 234ms/step - loss: 0.0049 - val_loss: 0.3611
Epoch 8/10
639/639 [==============================] - 151s 236ms/step - loss: 0.0044 - val_loss: 0.3239
Epoch 9/10
639/639 [==============================] - 149s 233ms/step - loss: 0.0043 - val_loss: 0.3095
Epoch 10/10
639/639 [==============================] - 151s 236m

In [43]:
# Load the best weights
model.load_weights("best_weights.h5")

Visualize prediction on Test Set

In [44]:
Predictions = model.predict(Xtest)

In [45]:
Predictions = Target_Scaler.inverse_transform(Predictions)
Actual = Target_Scaler.inverse_transform(Ytest)

In [46]:
Predictions.shape

(295, 1)

In [47]:
Predictions = np.squeeze(Predictions , axis = 1)
Actual = np.squeeze(Actual , axis = 1)

In [48]:
# Check the Predictions vs Actual
fig = go.Figure()

fig.add_trace(go.Scatter(x = data.index[-test_length:] , y = Actual , mode = 'lines' , name='Actual'))
fig.add_trace(go.Scatter(x = data.index[-test_length:] , y = Predictions , mode = 'lines' , name='Predicted'))
fig.show()

Visualize Prediction on whole data

In [49]:
Total_features = np.concatenate((Xtrain , Xtest) , axis = 0)

In [50]:
Total_Targets = np.concatenate((Ytrain , Ytest) , axis = 0)

In [51]:
Predictions = model.predict(Total_features)

In [52]:
Predictions = Target_Scaler.inverse_transform(Predictions)
Actual = Target_Scaler.inverse_transform(Total_Targets)

In [53]:
Predictions = np.squeeze(Predictions , axis = 1)
Actual = np.squeeze(Actual , axis = 1)

In [54]:
# Check the trend in Volume Traded
fig = go.Figure()

fig.add_trace(go.Scatter(x = data.index , y = Actual , mode = 'lines' , name='Actual'))
fig.add_trace(go.Scatter(x = data.index , y = Predictions , mode = 'lines' , name='Predicted'))
fig.show()

In [55]:
# Save and Load the whole model
model.save("Model.h5")
loaded_model = tf.keras.models.load_model("Model.h5")

Realtime Prediction

In [56]:
def PredictStockPrice(Model , DataFrame , PreviousDate , feature_length = 32):
    idx_location = DataFrame.index.get_loc(PreviousDate)
    Features = DataFrame.iloc[idx_location - feature_length : idx_location,:].values
    Features = np.expand_dims(Features , axis = 0)
    Features = Feature_Scaler.transform(Features)
    Prediction = Model.predict(Features)
    Prediction = Target_Scaler.inverse_transform(Prediction)
    return Prediction[0][0]

In [57]:
PredictStockPrice(loaded_model , data , '2021-01-14')

1705.5522